In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder
import transformers
import load_data
import torch
from torch import nn
from transformers import BertModel, BertTokenizer, CamembertModel, CamembertTokenizer
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AdamW, get_linear_schedule_with_warmup
from sklearn.metrics import f1_score
import numpy as np
import os
from utils_ipu import *
from training_text_model_ipu import *
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu') # GPU ou CPU
print(torch.cuda.is_available())
# Chargement des données
transcr_path='paco-cheese/transcr'
data=load_data.load_all_ipus(folder_path=transcr_path,load_words=True)



C:\Users\victo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True


In [2]:
data[275:285]

,ipu_id,speaker,start_ipu,stop_ipu,text_ipu,is_main_speaker,turn_at_start,turn_after,turn_start_word,yield_at_end,request_at_start,start_words,stop_words,duration,text_words,request_after_word,turn_after_word,is_ipu_end,dyad
275,33,AA,77.42,79.400,étendu devant la voiture il y avait un âne,True,False,True,79.4,True,False,78.78,78.89,0.11,y,False,False,False,transcr\AAOR
276,33,AA,77.42,79.400,étendu devant la voiture il y avait un âne,True,False,True,79.4,True,False,78.89,79.00,0.11,avait,False,False,False,transcr\AAOR
277,33,AA,77.42,79.400,étendu devant la voiture il y avait un âne,True,False,True,79.4,True,False,79.00,79.17,0.17,un,False,False,False,transcr\AAOR
278,33,AA,77.42,79.400,étendu devant la voiture il y avait un âne,True,False,True,79.4,True,False,79.17,79.40,0.23,âne,False,False,True,transcr\AAOR
279,34,OR,81.30,82.270,d'accord super,True,True,False,NaN,False,False,81.30,81.82,0.52,d'accord,False,False,False,transcr\AAOR
280,34,OR,81.30,82.270,d'accord super,True,True,False,NaN,False,False,82.00,82.27,0.27,super,False,False,True,transcr\AAOR
281,35,AA,83.64,84.120,voilà,False,False,False,NaN,False,False,83.64,84.12,0.48,voilà,False,False,True,transcr\AAOR
282,36,OR,86.76,89.382,mh attends j'avais plein de trucs à dire en mê...,True,False,False,NaN,False,True,86.76,87.59,0.83,mh,True,False,False,transcr\AAOR
283,36,OR,86.76,89.382,mh attends j'avais plein de trucs à dire en mê...,True,False,False,NaN,False,True,87.59,87.74,0.15,attends,False,False,False,transcr\AAOR
284,36,OR,86.76,89.382,mh attends j'avais plein de trucs à dire en mê...,True,False,False,NaN,False,True,87.74,87.83,0.09,j',False,False,False,transcr\AAOR


# creation des y à prédire

In [3]:

df = pd.DataFrame(data)


y=create_y(df) # Création de la target
df[:3]

,ipu_id,speaker,start_ipu,stop_ipu,text_ipu,is_main_speaker,turn_at_start,turn_after,turn_start_word,yield_at_end,request_at_start,start_words,stop_words,duration,text_words,request_after_word,turn_after_word,is_ipu_end,dyad
0,0,AA,4.54,4.840,tu as,True,False,True,4.84,True,False,4.54,4.72,0.18,tu,False,False,False,transcr\AAOR
1,0,AA,4.54,4.840,tu as,True,False,True,4.84,True,False,4.72,4.84,0.12,as,False,False,True,transcr\AAOR
2,1,OR,5.14,5.825,mh ouais si tu veux,True,True,False,NaN,False,True,5.14,5.44,0.30,mh,True,False,False,transcr\AAOR


In [4]:


features=create_sequences(df)
seed=42
X_train, X_test, y_train, y_test = train_test_split(features, y, test_size=0.2, random_state=seed)

# Vérification des dimensions
print(f"Nombre de séquences d'entraînement: {len(X_train)}, étiquettes: {len(y_train)}")
print(f"Nombre de séquences de test: {len(X_test)}, étiquettes: {len(y_test)}")


Nombre de séquences d'entraînement: 29160, étiquettes: 29160
Nombre de séquences de test: 7291, étiquettes: 7291


In [5]:
print(X_train)
print(y_train[365:375])

["m a i s   c ' e s t   u n   m e c   q u i   p a r e i l   t u   l u i   p o s e s   u n e   q u e s t i o n   i l   t e   d i t", "n o n   c ' é t a i t   p a s   ç a   q u e   j ' a v a i s", "e t   l ' a   d é n o n c é", "q u ' e s t - c e   q u ' i l   y   a", 't o u s   f a i t   a v a n t   o u a i s', "e n f i n   f r o g   j o k e   c ' é t a i t   u n e   v a n n e   h e i n   f r o g   j o k e", 'b o n n e   a m b i a n c e', 'p a r c e   q u e   j e   s a i s   p a s   j e   s u i s   p a s', "j e   p e n s e   p a s   q u ' o n   n o u s   a m è n e   c o m m e   ç a", "o u   d ' a u t e s   q u i   o n t   f a i t   c e   m a s t e r   e n   f a i t   t u   v o i s   p s k   c ' e s t   p a s   d e s", 'ç a   m   d i t   q u e q u e   c h o s e', "m a i s   ç a   c ' é t a i t   v r a i m e n t   l a   m a t i è r e   o ù   l à", "j e   s a i s   p l u s   v r a i m e n t   l ' i n t i t u l é   p a r c e   q u ' i l   a   c h a n g é   m a i s   c ' e s t", "s a   f e m

In [6]:
# Paramètres
model_name = 'camembert-base'
max_length = 256
batch_size = 16
epochs = 3
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Tokenizer pour CamemBERT
tokenizer = CamembertTokenizer.from_pretrained(model_name)

# Création des datasets et dataloaders
train_dataset = TextDataset(X_train, y_train, tokenizer, max_length)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = TextDataset(X_test, y_test, tokenizer, max_length)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
print(len(X_train))
print(len(y_train))

29160
29160


# Entrainement

In [8]:
from transformers import CamembertTokenizer, CamembertForSequenceClassification
# modèle CamemBERT pour la classification de séquences
model = CamembertForSequenceClassification.from_pretrained(model_name, num_labels=2)
#model=torch.load('modele/camembert_epoch_3.bin')
model.to(device)

model_save_path = 'modele'  # Chemin pour enregistrer le modèle
if not os.path.exists(model_save_path):
    os.makedirs(model_save_path)
    

# Optimiseur et taux d'apprentissage
optimizer = AdamW(model.parameters(), lr=2e-5)
total_steps = len(train_loader) * epochs
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

class_weights = torch.tensor([1.0, 2], device=device) # adaptation des poids des classes
loss_fn = nn.CrossEntropyLoss(weight=class_weights)

def train_epoch(model, data_loader, optimizer, device, scheduler, loss_fn):
    model.train()
    losses = []
    correct_predictions = 0

    for d in data_loader:

        input_ids = d["input_ids"].to(device)
        attention_mask = d["attention_mask"].to(device)
        labels = d["labels"].to(device)


        model.zero_grad()

   
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)


        loss = loss_fn(outputs.logits, labels)
        
        _, preds = torch.max(outputs.logits, dim=1)
        correct_predictions += torch.sum(preds == labels)
        losses.append(loss.item())
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()

    return correct_predictions.double() / len(data_loader.dataset), np.mean(losses)


# Boucle d'entraînement
for epoch in range(3):
    #epoch+=3
    print(f'Epoch {epoch + 1}/{epochs}')
    print('-' * 10)

    train_acc, train_loss = train_epoch(
        model,
        train_loader,
        optimizer,
        device,
        scheduler,
        loss_fn
    )

    print(f'Train loss {train_loss} accuracy {train_acc}')
    save_file_path = os.path.join(model_save_path, f'camembert_ipu_epoch_{epoch+1}.bin')
    torch.save(model, save_file_path)


Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\victo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/3
----------
Train loss 0.6173889159796595 accuracy 0.7634087791495199
Epoch 2/3
----------


KeyboardInterrupt: 

# score f1 et confusion matrix sur le test

In [9]:
from sklearn.metrics import f1_score, confusion_matrix
all_preds_text = []
all_labels = []
model= torch.load('modele/camembert_ipu_epoch_1.bin')
model.eval()
all_preds_text,all_labels = prediction_model_text(model,test_loader,device,proba=False)
f1 = f1_score(all_labels, all_preds_text)#, average='weighted')
conf_matrix = confusion_matrix(all_labels, all_preds_text)
print(f'Test F1 Score: {f1}')
print(f'Confusion Matrix:\n{conf_matrix}')

total_class_0 = np.sum(conf_matrix[0])
total_class_1 = np.sum(conf_matrix[1])
detected_class_0 = conf_matrix[0, 0]  # Vrais positifs pour la classe 0
detected_class_1 = conf_matrix[1, 1]  # Vrais positifs pour la classe 1

print(f'Nombre d\'éléments de classe 0 détectés : {detected_class_0} sur {total_class_0}')
print(f'Nombre d\'éléments de classe 1 détectés : {detected_class_1} sur {total_class_1}')


# prédictions sur quelques données afin de voir ce qu'il se passe : 

In [ ]:
import torch
from transformers import CamembertTokenizer

tokenizer = CamembertTokenizer.from_pretrained('camembert-base')

all_probs = []
data=X_train[273:277]
with torch.no_grad():
    for sentence in data:
        # Tokenize the sentence
        print(sentence)
        inputs = tokenizer(sentence, return_tensors='pt', padding=True, truncation=True, max_length=512)
        input_ids = inputs['input_ids'].to(device)
        attention_mask = inputs['attention_mask'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        
        probs = torch.softmax(logits, dim=1)
        all_probs.append(probs.cpu().numpy())



        print(outputs)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


voiture il y avait un âne
SequenceClassifierOutput(loss=None, logits=tensor([[-0.2287,  0.1516]], device='cuda:0'), hidden_states=None, attentions=None)
il y avait un âne d'accord
SequenceClassifierOutput(loss=None, logits=tensor([[-0.7145,  0.6228]], device='cuda:0'), hidden_states=None, attentions=None)
y avait un âne d'accord super
SequenceClassifierOutput(loss=None, logits=tensor([[-0.7334,  0.6410]], device='cuda:0'), hidden_states=None, attentions=None)
avait un âne d'accord super voilà
SequenceClassifierOutput(loss=None, logits=tensor([[-0.7127,  0.6209]], device='cuda:0'), hidden_states=None, attentions=None)
